In [1]:
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 27.2 MB/s eta 0:00:00


In [2]:
import torch
from torch.utils.data import DataLoader
from transformers import CamembertForTokenClassification, CamembertTokenizer
from jiwer import wer, cer
import matplotlib.pyplot as plt
import numpy as np
import json
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer
from torch.nn.utils.rnn import pad_sequence
import random
from tqdm.notebook import trange


In [3]:
# Charger les données depuis le fichier JSONL
data_train_path = '/content/drive/MyDrive/Colab Notebooks/TP_NLP/notes.jsonl'
data_val_path = '/content/drive/MyDrive/Colab Notebooks/TP_NLP/notes_validation.jsonl'

# Charger les données en tant que liste de dictionnaires
with open(data_train_path, 'r', encoding='utf-8') as file:
    data_train_list = [json.loads(line) for line in file]

with open(data_val_path, 'r', encoding='utf-8') as file:
    data_val_list = [json.loads(line) for line in file]

# Définir le pourcentage de données à conserver
percentage_to_keep = 0.025

# Sélectionner un sous-ensemble aléatoire
selected_train_data = random.sample(data_train_list, int(len(data_train_list) * percentage_to_keep))
selected_val_data = random.sample(data_val_list, int(len(data_val_list) * percentage_to_keep))

# Enregistrez le nouveau sous-ensemble dans un nouveau fichier JSONL
output_train_path = '/content/drive/MyDrive/Colab Notebooks/TP_NLP/notes_subset.jsonl'
with open(output_train_path, 'w', encoding='utf-8') as output_file:
    for item in selected_train_data:
        json.dump(item, output_file)
        output_file.write('\n')

output_val_path = '/content/drive/MyDrive/Colab Notebooks/TP_NLP/notes_validation_subset.jsonl'
with open(output_val_path, 'w', encoding='utf-8') as output_file:
    for item in selected_train_data:
        json.dump(item, output_file)
        output_file.write('\n')


In [4]:
# Charger les données depuis le fichier JSONL
data_train_path = '/content/drive/MyDrive/Colab Notebooks/TP_NLP/notes_subset.jsonl'
data_val_path = '/content/drive/MyDrive/Colab Notebooks/TP_NLP/notes_validation_subset.jsonl'

# Charger les données en tant que liste de dictionnaires
with open(data_train_path, 'r', encoding='utf-8') as file:
    data_train_list = [json.loads(line) for line in file]

with open(data_val_path, 'r', encoding='utf-8') as file:
    data_val_list = [json.loads(line) for line in file]

# Charger le tokenizer
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

class CorrectionDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=20):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text = "Corrige toutes les erreurs. " + self.data[idx]['input']
        target_text = self.data[idx]['output']

        # Prétraitement des données
        input_encoding = self.tokenizer(input_text, return_tensors='pt', max_length=self.max_length, padding=True, truncation=True)
        target_encoding = self.tokenizer(target_text, return_tensors='pt', max_length=self.max_length, padding=True, truncation=True)

        return {
            'input_ids': input_encoding['input_ids'].flatten(),
            'attention_mask': input_encoding['attention_mask'].flatten(),
            'labels': target_encoding['input_ids'].flatten()
        }
    def collate_fn(self, batch):
        input_ids = pad_sequence([item['input_ids'] for item in batch], padding_value=tokenizer.pad_token_id)
        attention_mask = pad_sequence([item['attention_mask'] for item in batch], padding_value=0)
        labels = pad_sequence([item['labels'] for item in batch], padding_value=tokenizer.pad_token_id)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

# Création de l'ensemble de données
dataset_train = CorrectionDataset(data_train_list, tokenizer)
dataset_val = CorrectionDataset(data_val_list, tokenizer)

# Création du DataLoader
batch_size = 4
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=dataset_train.collate_fn)
val_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle=True, collate_fn=dataset_val.collate_fn)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# Définir la fonction d'évaluation avec WER et CER
def evaluate(model, dataloader, tokenizer, device):
    model.eval()
    total_wer, total_cer = 0, 0
    total_samples = 0

    with torch.no_grad():
        for batch in dataloader:
            inputs = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=inputs, labels=labels, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)

            # Décodage des prédictions et des étiquettes
            decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            # Calcul des métriques WER et CER
            for pred, label in zip(decoded_preds, decoded_labels):
                if label != "":
                    total_wer += wer(label, pred)
                    total_cer += cer(label, pred)
                    total_samples += 1

    # Calcul des moyennes
    avg_wer = total_wer / max(1, total_samples)
    avg_cer = total_cer / max(1, total_samples)

    return avg_wer, avg_cer


In [8]:
# Charger le modèle T5 pré-entraîné
model_name = "google/flan-t5-base"
model = T5ForConditionalGeneration.from_pretrained(model_name)


tokenizer = T5Tokenizer.from_pretrained(model_name)

# Définir les hyperparamètres d'entraînement
epochs = 5
learning_rate = 1e-3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

# Définir la fonction de perte
criterion = torch.nn.CrossEntropyLoss()

# Définir l'optimiseur
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

print(model.config)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "

In [ ]:
# Entraîner le modèle avec évaluation périodique
train_losses = []
val_losses = []
wer_scores = []
cer_scores = []

for epoch in trange(epochs):
    model.train()
    total_train_loss = 0.0

    for batch in train_loader:
        inputs = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        attention_mask = batch['attention_mask'].to(device)


        # Calculer la perte
        outputs = model(input_ids=inputs, labels=labels, attention_mask=attention_mask)
        loss = outputs.loss

        # Rétropropagation et mise à jour des poids
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

    # Calculer la perte moyenne sur les données d'entraînement
    avg_train_loss = total_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Évaluer les performances sur les données de validation
    avg_wer, avg_cer = evaluate(model, val_loader, tokenizer, device)
    wer_scores.append(avg_wer)
    cer_scores.append(avg_cer)

    print(f"Epoch {epoch + 1}/{epochs} - Train Loss: {avg_train_loss:.4f} - WER: {avg_wer:.2%} - CER: {avg_cer:.2%}")

# Afficher la courbe de perte
plt.plot(train_losses, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Afficher les scores WER et CER au fil des époques
plt.plot(wer_scores, label='WER')
plt.plot(cer_scores, label='CER')
plt.xlabel('Epochs')
plt.ylabel('Score')
plt.legend()
plt.show()


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/5 - Train Loss: 6.1737 - WER: 100.40% - CER: 99.58%
